# Projet DataScience - Entreprise TouNum

## Méthodologie CRISP :

|Auteurs|Centre|Modification|Date|
|---|---|---|---|
|Alexis Billequin - Clément Chabrier - Etienne Duverney - Adrien Kegler - Nicolas Mazard - Baptiste Saclier - Adrien Thevenet|Lyon|première version|14/01/2020|

In [4]:
import pandas as pd

### Business Understanding :

#### Demande métier :

Nous travaillons pour TouNum, une entreprise de numérisation des cassettes vidéos.Le but de ce projet est d'améliorer les images de cassette vidéo VHS pour qu'elles soient analysables.

TouNum nous demande donc d'en un premier temps de travailler sur la restauration d'images [`PAL (Phase Alternating Line)`](https://www.wikiwand.com/fr/Phase_Alternating_Line) issu de cassettes VHS grâce au machine learning; Il faudra:
- Détecter/réduire/faire disparaitre les défauts
- Rendre le processus automatique
- Pouvoir s'adapter à tous les types de données d'images
- Rendre la solution lisible sous la forme de notebooks [`Jupyter`](https://jupyter.org/)
- Exporter le modèle de machine learning via [`Pickle`](https://docs.python.org/fr/3.7/library/pickle.html)
- En bonus:
    - Présenter des pistes de réflexions sur l'amélioration du modèle pour le traitement de vidéos

#### Evaluation du résultat :

Pour évaluer l'efficacité de notre solution, nous nous baserons sur une [`évaluation de la qualité d'image (EQI)`](https://www.researchgate.net/publication/322643565_Etude_comparative_des_methodes_d'evaluation_de_la_qualite_d'image). Il en existe trois catégorie:
- Les méthodes avec Référence: l'algorithme a un accès a une version parfaite de l'image avec laquelle il peut comparer la version déformée.
- Les méthodes sans Référence: l'algorithme n'a accès qu'à l'image déformée et doit estimer la qualité de l'image sans la connaissance de la version parfaite.
- Les méthodes avec Référence partielle: l'image de référence est disponible partiellement, elle est connu sous le nom d'estimation de la qualité de référence réduite

Il existe plusieur métrique d'**EQI**:
- L'**erreur moyenne quadratique** ou **Mean Square Error (MSE)** : comparer une image revalorisé avec l'image non dégrader, en se basant sur la MSE entre les pixels des deux images.
    $$MSE = \frac{1}{M*N} \sum_{m-1}^{M} \sum_{n-1}^{N}(I(m,n) - J(m,n)) $$ 
  $(M * N)$ est la taille de l'image, $I(m,n)$ et $J(m,n)$ sont les amplitudes des images parfaites et revalorisé  
  

- Le **rapport crête signal sur bruit** ou **Peak Signal Noise Ratio (PSNR)**: au lieu de mesurer les distorsion, cette valeur mesure la fidélité, c'est à dire qu'elle est proportionnelle à la qualité.
    $$ PSNR = 10 \log_{10}(\frac{{l_{max}}^2}{MSE}) $$
    $l_{max}$ désigne la luminance maximale possible pour une image en niveau de gris. Si cette valeur est infinie cela veut dire que l'image est non dégardée, et plus la valeur est dégradée plus cette valeur décroît.  
Dans le cas des images couleur, aucune définition précisede l’adaptation de l’équation du PSNR n’existe. Plusieursvariantes existent :
   - Le PSNR est calculé sur chacun des trois plans colorimétriques puis une moyenne est ensuite effectuée afin d'obtenir une valeur finie
   - La distance euclidienne entre deux pixels de couleur est utilisée dans l'équation originale
cette mesure ne permet pas de prendre en compte la sensibilité du système visuelhumain, et n’apparaît pas être performante.

- Le **rapport signal sur bruit** ou **Signal Noise Ratio (SNR)**: Variante du PSNR liant le MSE à l'énergie moyenne de l'image.
    $$ SNR = 10 \log_{10}(\frac{\frac{1}{N}\sum l^2}{MSE}) $$  


- L'**index de Similarité Structurelle (SSIM)**: est une mesure de similarité entre deux images numériques plustôt que la différence pixel à pixel.  
    La  métrique  SSIM  est  calculée  sur  plusieurs  fenêtres d'une image de taille NxN. On dénote x et y  les vecteurs de l’image originale et déformée respectivement. 
    
    $$SSIM(x, y) = \frac{(2\mu_x\mu_y+c_1)(\sigma_{xy} + c_2)}{(\mu_{x}^{2} + \mu_{y}^{2} + c_1)(\sigma_{x}^{2} + \sigma_{y}^{2} + c_2)}$$  
    
    La méthode SSIM est clairement plus impliquée que la méthode MSE, mais l'essentiel est que la méthode SSIM tente de modéliser le changement perçu dans les informations structurelles de l'image.   
    Les paramètres de l'équation comprennent l'emplacement (x, y) de la fenêtre N x N dans chaque image, la moyenne des intensités($\sigma$) des pixels dans les directions x et y, la variance des intensités ($\mu$) dans les directions x et y, ainsi que la covariance ($c_1$ et $c_2$) qui sont des constantes. La valeur de SSIM peut varier entre -1 et 1, où 1 indique une similarité parfaite.

In [10]:
pd.read_pickle("./score_method_comparision.pkl").style.background_gradient('RdYlGn')

,population,median_structural_similarity,median_dist_chebishev,median_dist_euclidean,median_dist_manathan,median_normalized_rmse
JPEG Artefact,601,0.688781,0.90763,0.885049,0.917426,0.867477
Tearing,574,0.702189,0.922161,0.894405,0.925426,0.881391
Posterisation,403,0.732297,0.928103,0.894176,0.912909,0.879247
Color Shifting,364,0.644015,0.904609,0.87221,0.899952,0.858027
Noise,334,0.629113,0.894139,0.873418,0.916455,0.870081
Blur,324,0.711261,0.918171,0.891501,0.922074,0.87593
Black Line,287,0.629452,0.87354,0.836486,0.870998,0.820096
Over-exposure,114,0.559261,0.252886,0.153862,0.553367,0.431939
White Line,107,0.749763,0.919153,0.907273,0.942537,0.881966
Pinking,89,0.692115,0.794987,0.784931,0.862725,0.717585


### Data Understanding :

A l'aide du dataSet fourni nous avons pu identifié 10 défauts sur les images:

* **Pinking** : Barre filtrante Magenta
* **Greening** : Barre filtrante Verte
* **Tearing** : Distortion, ligne déformante (I devient <)
* **Noise** : Bruis analogique (4 catégories types: gaussian - localvar - speckle - poisson)
* **Black Line** : Ligne noir fine horizontal
* **White Line** : Ligne blanche fine horizontal
* **Over-exposure** : Surexposition en luminausité
* **Blur**: traits des objets moins distincts
* **Color shifting** : Aberration chromatiques Rouges et Vertes
* **JPEG Artefact** : Artefactes de compression JPEG
* **Posterisation** : Réduction du nombre valeurs des dégradés

|Pinking|Greening|Tearing|Noise|Black Line|Posterisation|
|:-----:|:------:|:-----:|:---:|:--------:|:-----------:|
|![Pinking](../examples/pinking.png)|![Greening](../examples/greening.png)|![Tearing](../examples/tearing.png)|![Noise](../examples/noise.png)|![Black Line](../examples/black-line.png)|![Posterisation](../examples/posterisation.png)|
|**White Line**|**Over-exposure**|**Blur**|**Color Shifting**|**JPEG Artefact**|
|![White Line](../examples/white-line.png)|![Over-exposure](../examples/over-exposure.png)|![Blur](../examples/blur.png)|![Color Shifting](../examples/color-shifting.png)|![JPEG Artefact](../examples/jpeg-artefact.png)|

En observant la redondance des défauts nous avons défini une echelle de gravité de ces derniers sur les images avec la méthode de scoring la plus efficace la consernant:
1. **Over-exposure** $\Rightarrow$ median SSIM
2. **Noise** $\Rightarrow$ median distance manathan
3. **Blur** $\Rightarrow$ median distance manathan
4. **Posterization** $\Rightarrow$ median distance chebishev
5. **Greening/Pinking/Yellowing** $\Rightarrow$ median distance manathan
6. **Tearing** $\Rightarrow$ median distance manathan
7. **Color Shift** $\Rightarrow$ median distance chebishev
8. **Black/White Line** aussi appelé **scratch line** $\Rightarrow$ median distance manathan
9. **JPEG Artefacts** $\Rightarrow$ median distance manathan

### Data Preparation - Modeling - Evaluation :

Dans le cadre du développement d'une solution globale, nous nous sommes partager le travail en nous concentrant chacun sur un seul problème à la fois. 

Cette étape correspond à la création et préparation des données pour pouvoir par la suite entrainer un modèle de détection et de réparation d'une des erreur 

#### 1. Noise

Les images numérisées depuis un support analogiques présentent un certain grain. Ce grain (appelé Noise) peut être corrigé par l'algorithme développé dans le Notebook `noise-filter.ipynb`.

#### 2. Pinking/Greening

Certaines images numérisées depuis un support analogiques présentent une bande presque monochromatique dû à une perte de niveaux de verts,rouge ou bleu (RGB) sur l'image VHS avec le viellissement du matériel. 

Une explication plus pousée peut être trouvée dans le Notebook `coloring.ipynb`.

#### 3. Color shift

Une aberration chromatique est un efet d'optique qui produit différentes mises au point en fonction de la longueur d'onde. On observe alors une image floue et aux contours irisés. Elle résulte de la décomposition de la lumière blanche en plusieurs bandes de couleurs. Elle est reconnaissable à l'aura magenta/vert présent autour des contour de certains objet sur une image.

Une explication plus pousée peut être trouvée dans le fichier `Color_Shift.ipnyb`.

#### 4. Tearing

L'explication de la démarche pour détecter et réparer les décalages de quelques pixels est décrite dans le notebook `Tearing_reconnaissance.ipnyb`.

#### 5. Black/White Line

Les rayures de lignes sont un problème courant dans les films archivés. Le problème est transféré sur la vidéo pendant le processus de transfert du télécinéma.  Elle peut être causée par le frottement de la matière d'une particule sur le matériau du film dans le projecteur ou par l'abrasion du film lorsqu'il passe sur une particule prise dans le mécanisme.

L'explication de la démarche pour détecter et réparer les black ou white line est décrite dans le notebook `black_white_line.ipnyb`


#### 6. Blur

Tout est expliqué dans le fichier `report_blur.ipnyb`. Avec l'ensemble des étapes, l'installation de chaque librairies et l'utilisation de plusieurs algorithme. Malheureusement la doc de la classe finale n'est pas complète. Les noms des fonctions sont compréhensibles.

#### 7. Modèle Globale

![Architecture Globale](../examples/architecture_model.png)

En ce qui concerne l'architecture globale nous en avons définit 2 architectures qui nous paraissaient correspondre au problème.

Npus avons choisis de mettre en place la première architecture afin de détecter le maximum de problème et de pouvoir les régler.

### Deployment:

### Conclusion :

Pour que ce model puisse s'appliquer à des vidéos, nous avons trouver des fonctionna